In [5]:
import requests
import pandas as pd
import xml.etree.ElementTree as et
from itertools import chain
from collections import defaultdict
from pathlib import Path

In [6]:
data_dir = Path('../pathways/parsed')
all_pathways = pd.read_csv(data_dir.parent / 'pathway_manifest.csv', dtype='str')

In [7]:
def trim_ec(ec):
    return ec[3:] if ec.startswith('ec:') else ec

def read_csv_or_none(f_path):
    try:
        return pd.read_csv(f_path, converters={'source': trim_ec, 'target': trim_ec, 'pathway_number': str})
    except FileNotFoundError:
        return None

for superpathway, chunk in all_pathways.groupby('superpathway'):
    dfs = [
        df for d in chunk.to_dict('records')
        if (df:=read_csv_or_none(data_dir / f"{d['number']}.csv")) is not None
    ]
    if len(dfs) > 0:
        super_df = pd.concat(dfs)
        super_df['pathway_number'] = super_df['pathway_number'].astype(str)
        super_df.to_csv(data_dir.parent / f"{superpathway.lower().replace(' ', '_')}.csv", index=False)

In [8]:
super_df

,source,target,pathway_number
0,1.14.13.33,1.13.11.3,00362
1,1.14.13.2,1.13.11.3,00362
2,1.14.13.23,1.13.11.3,00362
3,1.14.13.24,,00362
4,1.14.13.64,1.13.11.37,00362
...,...,...,...
34,1.11.2.2,,00983
35,2.3.1.5,2.3.1.5,00983
36,3.1.1.1,,00983
37,3.1.1.1,2.3.1.5,00983
